In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('model-selection-joined.csv', index_col = 0)

/Users/bigyanbhandari/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/bigyanbhandari/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df.head()

,SCHOOL,STATE,ABBR,ID,COUNTY,CHARTER,MAGNET,LOCALE,TITLE I,TITLE I ELIGIBLE,...,MALE STUDENTS,FEMALE STUDENTS,AMERICAN INDIAN,ASIAN,HISPANIC,BLACK,WHITE,TOTAL RACE/ETHINICITY,FTE TEACHERS,PUPIL-TEACHER RATIO
0,101 ELEMENTARY ...,California,CA,6.410401e+10,SAN BERNARDINO,1-Yes,2-No,3-Urban fringe of large city,†,2-No,...,38,49,0,1,24,17,45,87,0,0
1,3R MIDDLE/COMMUNITY,California,CA,6.910231e+10,NEVADA,2-No,2-No,6-Small town,†,2-No,...,13,1,0,0,0,0,14,14,1,14
2,4-J ELEMENTARY SCHOOL,Wyoming,WY,5.601470e+11,CAMPBELL,2-No,2-No,6-Small town,2-No,1-Yes,...,29,22,0,0,2,0,49,51,0,0
3,5TH AND 6TH GRADE CTR.,Missouri,MO,2.928260e+11,SCOTT,2-No,2-No,6-Small town,†,2-No,...,345,304,4,2,2,168,473,649,0,0
4,6TH GRADE CENTER,Texas,TX,4.821720e+11,HUNT,2-No,–,3-Urban fringe of large city,1-Yes,1-Yes,...,388,404,3,4,116,196,473,792,49,0


In [4]:
df.columns

Index(['SCHOOL', 'STATE', 'ABBR', 'ID', 'COUNTY', 'CHARTER', 'MAGNET',
       'LOCALE', 'TITLE I', 'TITLE I ELIGIBLE', 'TOTAL STUDENTS',
       'FREE LUNCH ELIGIBLE', 'REDUCED PRICE LUNCH ELIGIBLE',
       'FREE AND REDUCED LUNCH ELIGIBLE', 'MALE STUDENTS', 'FEMALE STUDENTS',
       'AMERICAN INDIAN', 'ASIAN', 'HISPANIC', 'BLACK', 'WHITE',
       'TOTAL RACE/ETHINICITY', 'FTE TEACHERS', 'PUPIL-TEACHER RATIO'],
      dtype='object')

In [5]:
df.shape

(1428355, 24)

In [6]:
df.drop(columns = ['SCHOOL', 'STATE', 'ID', 'COUNTY', 'CHARTER', 'MAGNET', 'TITLE I',
                   'REDUCED PRICE LUNCH ELIGIBLE', 'FREE AND REDUCED LUNCH ELIGIBLE',
                  'MALE STUDENTS', 'FEMALE STUDENTS', 'TOTAL RACE/ETHINICITY', 'FTE TEACHERS', 'PUPIL-TEACHER RATIO'], inplace = True)

In [7]:
df.reset_index( drop = True, inplace = True)

Function to Drop invalid free lunch eligible students.

In [21]:
invalid_total_students = df['TOTAL STUDENTS'].apply( lambda x : x if ( (str(x)).isdigit() and x != 0) else None)

In [22]:
invalid_total_students = invalid_total_students[invalid_total_students.isna() == True].index

In [23]:
df.drop(invalid_total_students, inplace = True)

In [24]:
df.reset_index(inplace = True, drop = True)

In [25]:
invalid_idx = df['FREE LUNCH ELIGIBLE'].apply( lambda x : None if (not str(x).isdigit() and df['FREE LUNCH ELIGIBLE'] > df['TOTAL STUDENTS']) else x)

In [26]:
invalid_idx = invalid_idx[invalid_idx.isna() == True].index

In [27]:
df.drop(invalid_idx, inplace = True)

In [28]:
df.shape

(1420449, 10)

# Adding Label

In [31]:
poverty = df['FREE LUNCH ELIGIBLE']/df['TOTAL STUDENTS']

In [32]:
hi_poverty = poverty.apply( lambda x : 1 if x > 0.233 else 0)

In [33]:
strict_poverty = poverty.apply( lambda x : 1 if x > 0.339 else 0)

In [41]:
hi_poverty.groupby(hi_poverty).count()

0    483661
1    936788
dtype: int64

In [42]:
strict_poverty.groupby(strict_poverty).count()

0    690037
1    730412
dtype: int64

In [43]:
df['HI-LABEL'] = hi_poverty
df['STRICT-LABEL'] = strict_poverty

In [44]:
df.head()

,ABBR,LOCALE,TITLE I ELIGIBLE,TOTAL STUDENTS,FREE LUNCH ELIGIBLE,AMERICAN INDIAN,ASIAN,HISPANIC,BLACK,WHITE,HI-LABEL,STRICT-LABEL
0,CA,3-Urban fringe of large city,2-No,87,0,0,1,24,17,45,0,0
1,CA,6-Small town,2-No,14,0,0,0,0,0,14,0,0
2,WY,6-Small town,1-Yes,51,12,0,0,2,0,49,1,0
3,MO,6-Small town,2-No,649,272,4,2,2,168,473,1,1
4,TX,3-Urban fringe of large city,1-Yes,792,333,3,4,116,196,473,1,1


In [ ]:
df.drop( columns = 'TITLE I ELIGIBLE', 'FREE LUNCH ELIGIBLE', 'AMERICAN INDIAN')

In [45]:
#df.to_csv('data_for_training.csv')